# Import Libraries

In [18]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from transformers import pipeline
import numpy as np

# Query data from CSV

In [2]:
# Query from CSV
df_enriched_stock_data = pd.read_csv('/Users/ani/Projects/6_stock_portfolio_recommendation/data//enriched_stock_data.csv')
df_enriched_stock_data

,Ticker,Closing_Price,All_Time_High,Percent_From_All_Time_High,Percent_Difference_200_Day_Moving_Average,24_Hour_Percent_Change,7_Day_Percent_Change,30_Day_Percent_Change,Annualized_Return,YTD_Return,...,Industry,Country,Business_Summary,Dividend_Yield,Trailing_PE,Forward_PE,Average_Volume,Average_Volume_10days,52_Week_Change,Update_Date
0,NVDA,157.75,157.75,0.00,22.11,1.76,9.46,16.57,60.67,14.07,...,Semiconductors,United States,"NVIDIA Corporation, a computing infrastructure...",0.03,50.887100,38.288837,249322685,187835800,0.254816,2025-06-27
1,MSFT,495.94,497.45,-0.30,17.61,-0.30,3.74,9.69,17.33,18.94,...,Software - Infrastructure,United States,Microsoft Corporation develops and supports so...,0.67,38.355762,33.173244,23145869,20812190,0.112988,2025-06-27
2,AAPL,201.08,258.40,-22.18,-9.79,0.04,2.78,-5.30,14.78,-17.34,...,Consumer Electronics,United States,"Apple Inc. designs, manufactures, and markets ...",0.52,31.369736,24.197351,61975983,51950890,-0.045675,2025-06-27
3,AMZN,223.30,242.06,-7.75,8.96,2.85,3.95,6.21,8.88,1.40,...,Internet Retail,United States,"Amazon.com, Inc. engages in the retail sale of...",0.00,36.368080,36.308945,49333604,39989330,0.123519,2025-06-27
4,GOOGL,178.53,205.89,-13.29,3.89,2.88,1.47,8.09,17.04,-5.53,...,Internet Content & Information,United States,Alphabet Inc. offers various products and plat...,0.48,19.947487,19.925222,41357416,37073840,-0.047269,2025-06-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,IVZ,15.70,24.58,-36.12,-2.56,0.77,8.13,1.42,11.01,-8.82,...,Asset Management,United States,Invesco Ltd. is a publicly owned investment ma...,5.39,12.559999,8.306878,5714119,5047610,0.041444,2025-06-27
496,APA,18.56,46.75,-60.30,-10.65,0.60,-10.68,2.15,7.67,-18.45,...,Oil & Gas E&P,United States,"APA Corporation, an independent energy company...",5.42,6.652329,6.749091,9028964,10513290,-0.373302,2025-06-27
497,MHK,104.90,229.74,-54.34,-15.38,1.50,6.57,-2.20,0.83,-9.51,...,"Furnishings, Fixtures & Appliances",United States,"Mohawk Industries, Inc. designs, manufactures,...",0.00,13.712419,9.408072,803162,920930,-0.090149,2025-06-27
498,CZR,28.86,119.49,-75.85,-14.37,1.23,9.11,-5.96,-5.13,-11.45,...,Resorts & Casinos,United States,"Caesars Entertainment, Inc. operates as a gami...",0.00,0.000000,21.537313,5763859,5734720,-0.282587,2025-06-27


# Define Functions

In [3]:
news_api_key = open("/Users/ani/Documents/0_API_KEYS/newsapi.txt").read().strip()
base_url = "https://newsapi.org/v2/everything"
sentiment_analyzer = pipeline(
                "sentiment-analysis",
                model="ProsusAI/finbert",
                tokenizer="ProsusAI/finbert"
            )

Device set to use mps:0


In [ ]:
def get_news_articles(ticker_symbol,  api_key, base_url, company_name= None, 
                        days_back=7, page_size= 100):
    """
    Fetch news articles for a given stock ticker.
    
    Args:
        ticker_symbol (str): Stock ticker symbol (e.g., 'AAPL')
        company_name (str): Company name for better search results
        days_back (int): Number of days to look back for news
        page_size (int): Number of articles to fetch (max 100)
        
    Returns:
        List[Dict]: List of news articles
    """
    # Calculate date range
    to_date = datetime.now()
    from_date = to_date - timedelta(days=days_back)
    
    # Construct search query
    if company_name:
        query = f'"{ticker_symbol}" OR "{company_name}"'
    else:
        query = f'"{ticker_symbol}"'
    
    # API parameters
    params = {
        'q': query,
        'apiKey': api_key,
        'language': 'en',
        'sortBy': 'relevancy',
        'pageSize': min(page_size, 100),
        'from': from_date.strftime('%Y-%m-%d'),
        'to': to_date.strftime('%Y-%m-%d')
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        data = response.json()
        
        if data['status'] == 'ok':
            articles = data.get('articles', [])
            return articles
        else:
            return []
            
    except requests.exceptions.RequestException as e:
        return []
    except Exception as e:
        return []

In [5]:
articles = get_news_articles('NVDA',  news_api_key, base_url, company_name= None, 
                        days_back=7, page_size= 100)
articles[:2]

429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22NVDA%22&apiKey=a101780fac934cd6bf8fad7ddff97331&language=en&sortBy=relevancy&pageSize=100&from=2025-06-22&to=2025-06-29


[]

In [6]:
def analyze_text_sentiment(text, sentiment_analyzer):
    """
    Analyze sentiment of a single text using HuggingFace model.
    
    Args:
        text (str): Text to analyze
        
    Returns:
        Dict: Sentiment analysis results
    """
    try:
        # Truncate text if too long (BERT models have token limits)
        max_length = 512
        if len(text.split()) > max_length:
            text = ' '.join(text.split()[:max_length])
        
        # Get sentiment prediction
        result = sentiment_analyzer(text)[0]
        
        # Normalize labels (different models use different labels)
        label = result['label'].upper()
        score = result['score']
        
        # Map different model outputs to consistent labels
        if 'Positive' in label or 'POS' in label:
            sentiment = 'Positive'
        elif 'NEGATIVE' in label or 'NEG' in label:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'
        
        return {
            'sentiment': sentiment,
            'confidence': score,
            'raw_label': result['label']
        }
        
    except Exception as e:
        return {
            'sentiment': 'Neutral',
            'confidence': 0.0,
            'raw_label': 'error'
        }

In [7]:
analyze_text_sentiment('JPMorgan cuts Apple stock price target on soft iPhone 17 outlook and delayed AI payoff', sentiment_analyzer)

{'sentiment': 'Negative',
 'confidence': 0.9300100803375244,
 'raw_label': 'negative'}

In [8]:
def analyze_articles_sentiment(articles, sentiment_analyzer):
    """
    Analyze sentiment for a list of news articles.
    
    Args:
        articles (List[Dict]): List of news articles
        
    Returns:
        Dict: Aggregated sentiment analysis
    """
    if not articles:
        return {
            'Sentiment': 'Neutral',
            # 'Sentiment_Score': 0.0,
            # 'positive_count': 0,
            # 'negative_count': 0,
            # 'neutral_count': 0,
            # 'total_articles': 0,
            # 'confidence': 0.0
        }
    
    sentiments = []
    sentiment_scores = []
    
    for article in articles:
        # Combine title and description for analysis
        text_to_analyze = ""
        if article.get('title'):
            text_to_analyze += article['title'] + " "
        if article.get('description'):
            text_to_analyze += article['description']
        
        if text_to_analyze.strip():
            sentiment_result = analyze_text_sentiment(text_to_analyze, sentiment_analyzer)
            sentiments.append(sentiment_result['sentiment'])
            
            # Convert sentiment to numerical score
            if sentiment_result['sentiment'] == 'Positive':
                score = sentiment_result['confidence']
            elif sentiment_result['sentiment'] == 'Negative':
                score = -sentiment_result['confidence']
            else:
                score = 0.0
            
            sentiment_scores.append(score)
    
    # Calculate aggregated metrics
    positive_count = sentiments.count('Positive')
    negative_count = sentiments.count('negative')
    neutral_count = sentiments.count('Neutral')
    total_articles = len(sentiments)
    
    # Calculate overall sentiment score (average)
    overall_score = np.mean(sentiment_scores) if sentiment_scores else 0.0
    
    # Determine overall sentiment
    if overall_score > 0.1:
        Sentiment = 'Positive'
    elif overall_score < -0.1:
        Sentiment = 'negative'
    else:
        Sentiment = 'Neutral'
    
    # Calculate confidence as standard deviation (lower = more confident)
    confidence = 1 - (np.std(sentiment_scores) if len(sentiment_scores) > 1 else 0)
    confidence = max(0, min(1, confidence))  # Clamp between 0 and 1
    
    return {
        'Sentiment': Sentiment
    }

In [9]:
analyze_articles_sentiment(articles, sentiment_analyzer)

{'Sentiment': 'Neutral'}

In [10]:

def get_stock_sentiment(ticker_symbol):
    """
    Get complete sentiment analysis for a stock.
    
    Args:
        ticker_symbol (str): Stock ticker symbol
        company_name (str): Company name (optional)
        
    Returns:
        Dict: Complete sentiment analysis results
    """
    
    # Fetch news articles
    articles = get_news_articles(ticker_symbol,  news_api_key, base_url, 
                        days_back=7, page_size= 100)
    
    # Analyze sentiment
    sentiment_results = analyze_articles_sentiment(articles, sentiment_analyzer)
    
    # Add metadata
    sentiment_results.update({
        'Ticker': ticker_symbol,
    })
    
    return sentiment_results


In [11]:
get_stock_sentiment('AAPL')

429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22AAPL%22&apiKey=a101780fac934cd6bf8fad7ddff97331&language=en&sortBy=relevancy&pageSize=100&from=2025-06-22&to=2025-06-29


{'Sentiment': 'Neutral', 'Ticker': 'AAPL'}

In [12]:
def analyze_multiple_stocks(tickers):
    """
    """


    results = []
    for ticker in tickers:
        try:
            sentiment_results = get_stock_sentiment(ticker)
            results.append(sentiment_results)

            time.sleep(0.1)
        
        except Exception as e:
            results.append({
                'Ticker': ticker,
                'Sentiment': 'Neutral',
            })

    return pd.DataFrame(results)

In [13]:
df_sentiment = analyze_multiple_stocks(df_enriched_stock_data['Ticker'].unique().tolist())
df_sentiment

429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22NVDA%22&apiKey=a101780fac934cd6bf8fad7ddff97331&language=en&sortBy=relevancy&pageSize=100&from=2025-06-22&to=2025-06-29
429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22MSFT%22&apiKey=a101780fac934cd6bf8fad7ddff97331&language=en&sortBy=relevancy&pageSize=100&from=2025-06-22&to=2025-06-29
429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22AAPL%22&apiKey=a101780fac934cd6bf8fad7ddff97331&language=en&sortBy=relevancy&pageSize=100&from=2025-06-22&to=2025-06-29
429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22AMZN%22&apiKey=a101780fac934cd6bf8fad7ddff97331&language=en&sortBy=relevancy&pageSize=100&from=2025-06-22&to=2025-06-29
429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22GOOGL%22&apiKey=a101780fac934cd6bf8fad7ddff97331&language=en&sortBy=relevancy&pageSize=100&from=2025-06-22&to=2025-06-29
429 Client Error:  for url: https://newsapi.org/v2/everything?q=

,Sentiment,Ticker
0,Neutral,NVDA
1,Neutral,MSFT
2,Neutral,AAPL
3,Neutral,AMZN
4,Neutral,GOOGL
...,...,...
495,Neutral,IVZ
496,Neutral,APA
497,Neutral,MHK
498,Neutral,CZR


# Add sentiment to dataframe

In [14]:
df_enriched_stock_data_w_sentiment = pd.merge(df_enriched_stock_data, df_sentiment, how='left', on=['Ticker'])
df_enriched_stock_data_w_sentiment

,Ticker,Closing_Price,All_Time_High,Percent_From_All_Time_High,Percent_Difference_200_Day_Moving_Average,24_Hour_Percent_Change,7_Day_Percent_Change,30_Day_Percent_Change,Annualized_Return,YTD_Return,...,Country,Business_Summary,Dividend_Yield,Trailing_PE,Forward_PE,Average_Volume,Average_Volume_10days,52_Week_Change,Update_Date,Sentiment
0,NVDA,157.75,157.75,0.00,22.11,1.76,9.46,16.57,60.67,14.07,...,United States,"NVIDIA Corporation, a computing infrastructure...",0.03,50.887100,38.288837,249322685,187835800,0.254816,2025-06-27,Neutral
1,MSFT,495.94,497.45,-0.30,17.61,-0.30,3.74,9.69,17.33,18.94,...,United States,Microsoft Corporation develops and supports so...,0.67,38.355762,33.173244,23145869,20812190,0.112988,2025-06-27,Neutral
2,AAPL,201.08,258.40,-22.18,-9.79,0.04,2.78,-5.30,14.78,-17.34,...,United States,"Apple Inc. designs, manufactures, and markets ...",0.52,31.369736,24.197351,61975983,51950890,-0.045675,2025-06-27,Neutral
3,AMZN,223.30,242.06,-7.75,8.96,2.85,3.95,6.21,8.88,1.40,...,United States,"Amazon.com, Inc. engages in the retail sale of...",0.00,36.368080,36.308945,49333604,39989330,0.123519,2025-06-27,Neutral
4,GOOGL,178.53,205.89,-13.29,3.89,2.88,1.47,8.09,17.04,-5.53,...,United States,Alphabet Inc. offers various products and plat...,0.48,19.947487,19.925222,41357416,37073840,-0.047269,2025-06-27,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,IVZ,15.70,24.58,-36.12,-2.56,0.77,8.13,1.42,11.01,-8.82,...,United States,Invesco Ltd. is a publicly owned investment ma...,5.39,12.559999,8.306878,5714119,5047610,0.041444,2025-06-27,Neutral
496,APA,18.56,46.75,-60.30,-10.65,0.60,-10.68,2.15,7.67,-18.45,...,United States,"APA Corporation, an independent energy company...",5.42,6.652329,6.749091,9028964,10513290,-0.373302,2025-06-27,Neutral
497,MHK,104.90,229.74,-54.34,-15.38,1.50,6.57,-2.20,0.83,-9.51,...,United States,"Mohawk Industries, Inc. designs, manufactures,...",0.00,13.712419,9.408072,803162,920930,-0.090149,2025-06-27,Neutral
498,CZR,28.86,119.49,-75.85,-14.37,1.23,9.11,-5.96,-5.13,-11.45,...,United States,"Caesars Entertainment, Inc. operates as a gami...",0.00,0.000000,21.537313,5763859,5734720,-0.282587,2025-06-27,Neutral


# Save to CSV

In [17]:
df_enriched_stock_data_w_sentiment.isna().sum()

Ticker                                       0
Closing_Price                                0
All_Time_High                                0
Percent_From_All_Time_High                   0
Percent_Difference_200_Day_Moving_Average    0
24_Hour_Percent_Change                       0
7_Day_Percent_Change                         0
30_Day_Percent_Change                        0
Annualized_Return                            0
YTD_Return                                   0
2024_Return                                  0
2023_Return                                  0
2022_Return                                  0
2021_Return                                  0
2020_Return                                  0
Annualized_Volatility                        0
Sharpe_Ratio                                 0
Beta                                         0
Company_Name                                 0
Market_Cap                                   0
Sector                                       0
Industry     

In [15]:
assert df_enriched_stock_data_w_sentiment.isna().sum().sum() == 0, "DataFrame contains NaN values"

In [16]:
df_enriched_stock_data_w_sentiment.to_csv('/Users/ani/Projects/6_stock_portfolio_recommendation/data/enriched_stock_data_w_sentiment.csv', index=False)
print("Enriched stock data saved to 'data/enriched_stock_data_w_sentiment.csv'")

Enriched stock data saved to 'data/enriched_stock_data_w_sentiment.csv'
